In [2]:
import pandas as pd
import sys  
sys.path.insert(0, '../scripts')
import utilities
import seaborn as sn
import matplotlib.pyplot as plt
# import norm_scaling
import loading_data

In [3]:
# load data
df = loading_data.load_csv('../Data/AdSmartABdata.csv')

/home/dibora/Testing-A-B/ab_testing/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  


### Split data by browser and platform_os, and version each split as a new version of the data in dvc.


In [7]:
df_browser = df.groupby("browser")
df_os = df.groupby("platform_os")

## Machine Learning for A/B Testing
Using Machine Learning for A/B Testing allows to consider different features when doing our analysis.

In [ ]:
# Split the data into 70% training, 20% validation, and 10% test sets. 


In [ ]:
#  Train a machine learning model using 5-fold cross validation using the following 3 different algorithms:
# Logistic Regression 
# Decision Trees
# XGBoost


In [ ]:
Define the appropriate loss function  for the model using the validation data. 


In [ ]:
Compute feature importance - what’s driving the model? Which parameters are important predictors for the different ML models? What contributes to the goal of gaining more “Yes” results?
Which data features are relevant to predicting the target variable?



In [ ]:
For each of the ML algorithms above, find the best model by tuning their hyperparameters and each time adding the tried models in MLFlow.
Prepare a Dockerfile for your project so that your model can be deployed in a docker container.